In [ ]:
import torch
import numpy as np
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

In [ ]:
lr = 0.001
kapa = 0.0057 # values obtained from theoretical model
tao = 0.0013
diffl = 157.4158
layers = np.array([1, 100, 100, 100, 100, 100, 4])
N_f = 2000
N_u = 1
N_t = 100
steps = 42000

In [ ]:
kapa = torch.tensor([kapa], requires_grad = True).float().to(device)
tao = torch.tensor([tao], requires_grad = True).float().to(device)

In [ ]:
class FCN(torch.nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.activation = torch.nn.Tanh()
        self.loss_function = torch.nn.MSELoss(reduction = "mean")
        self.linears1 = torch.nn.ModuleList([torch.nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.linears2 = torch.nn.ModuleList([torch.nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.linears3 = torch.nn.ModuleList([torch.nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        
        self.iter = 0
        for i in range(len(layers)-1):
            torch.nn.init.xavier_normal_(self.linears1[i].weight.data, gain=1.0)
            torch.nn.init.zeros_(self.linears1[i].bias.data)
            torch.nn.init.xavier_normal_(self.linears2[i].weight.data, gain=1.0)
            torch.nn.init.zeros_(self.linears2[i].bias.data)
            torch.nn.init.xavier_normal_(self.linears3[i].weight.data, gain=1.0)
            torch.nn.init.zeros_(self.linears3[i].bias.data)
        
        self.kapa = torch.nn.Parameter(kapa)
        self.tao = torch.nn.Parameter(tao)
        
    def forward1(self, x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
        x = (x - l_b)/(u_b - l_b)
        a = x.float()
        
        for i in range(len(layers)-3):
            identity = a
            z = self.linears1[i](a)
            # z = z + identity
            a = torch.cos(z)
            a = a + identity
        identity = a
        z = self.linears1[-2](a)
        # z = z + identity
        a = torch.cos(z)
        a = a + identity
        a = self.linears1[-1](a)
        
        return a
    
    def forward2(self, x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
        x = (x - l_b)/(u_b - l_b)
        a = x.float()
        
        for i in range(len(layers)-3):
            identity = a
            z = self.linears2[i](a)
            # z = z + identity
            a = torch.sin(z)
            a = a + identity
        identity = a
        z = self.linears2[-2](a)
        # z = z + identity
        a = torch.sin(z)
        a = a + identity
        a = self.linears2[-1](a)
        
        return a
    
    def forward3(self, x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
        x = (x - l_b)/(u_b - l_b)
        a = x.float()
        
        for i in range(len(layers)-2):
            identity = a
            z = self.linears3[i](a)
            # z = z + identity
            a = self.activation(z)
            a = a + identity
        a = self.linears3[-1](a)
        
        return a
    
    def loss_data(self, X_data, U_data):
        x = self.forward1(X_data)
        y = self.forward2(X_data)
        z = self.forward3(X_data)
        
        x_error = self.loss_function(x[:, 0:1], U_data[:, 0:1])
        y_error = self.loss_function(y[:, 0:1], U_data[:, 1:2])
        z_error = self.loss_function(z[:, 0:1], U_data[:, 2:3])
        
        loss_d = x_error + y_error + z_error
        
        return loss_d
    
    def loss_BC(self, X_train_Nu, U_train_Nu):
        x = self.forward1(X_train_Nu).reshape((1, 4))
        y = self.forward2(X_train_Nu).reshape((1, 4))
        z = self.forward3(X_train_Nu).reshape((1, 4))
        loss_u1 = self.loss_function(x[:, 0:1], U_train_Nu[:, 0:1])
        loss_u2 = self.loss_function(y[:, 0:1], U_train_Nu[:, 1:2])
        loss_u3 = self.loss_function(z[:, 0:1], U_train_Nu[:, 2:3])
        
        loss_u = loss_u1 + loss_u2 + loss_u3
        
        return loss_u
    
    def loss_ODE(self, X_train_Nf):
        kapa = self.kapa
        tao = self.tao
        
        g = X_train_Nf.clone()
        g.requires_grad = True
        
        x = self.forward1(g) #[1001, 4]
        y = self.forward2(g)
        z = self.forward3(g)
        
        x1_grad = torch.autograd.grad(x[:, 0:1], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        x2_grad = torch.autograd.grad(x[:, 1:2], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        x3_grad = torch.autograd.grad(x[:, 2:3], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        x4_grad = torch.autograd.grad(x[:, 3:4], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        
        y1_grad = torch.autograd.grad(y[:, 0:1], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        y2_grad = torch.autograd.grad(y[:, 1:2], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        y3_grad = torch.autograd.grad(y[:, 2:3], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        y4_grad = torch.autograd.grad(y[:, 3:4], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        
        z1_grad = torch.autograd.grad(z[:, 0:1], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        z2_grad = torch.autograd.grad(z[:, 1:2], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        z3_grad = torch.autograd.grad(z[:, 2:3], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        z4_grad = torch.autograd.grad(z[:, 3:4], g, torch.ones([X_train_Nf.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        
        f1 = x1_grad - y[:, 1:2] * diffl
        f2 = y1_grad - x[:, 1:2] * diffl
        f3 = z1_grad - z[:, 1:2] * diffl
        f4 = x2_grad - kapa * y[:, 2:3] * diffl
        f5 = y2_grad - kapa * x[:, 2:3] * diffl
        f6 = z2_grad - kapa * z[:, 2:3] * diffl
        f7 = x3_grad - (- kapa * y[:, 1:2] + tao * y[:, 3:4]) * diffl
        f8 = y3_grad - (- kapa * x[:, 1:2] + tao * x[:, 3:4]) * diffl
        f9 = z3_grad - (- kapa * z[:, 1:2] + tao * z[:, 3:4]) * diffl
        f10 = x4_grad + tao * y[:, 2:3] * diffl
        f11 = y4_grad + tao * x[:, 2:3] * diffl
        f12 = z4_grad + tao * z[:, 2:3] * diffl
        
        loss_f1 = self.loss_function(f1, f_hat)
        loss_f2 = self.loss_function(f2, f_hat)
        loss_f3 = self.loss_function(f3, f_hat)
        loss_f4 = self.loss_function(f4, f_hat)
        loss_f5 = self.loss_function(f5, f_hat)
        loss_f6 = self.loss_function(f6, f_hat)
        loss_f7 = self.loss_function(f7, f_hat)
        loss_f8 = self.loss_function(f8, f_hat)
        loss_f9 = self.loss_function(f9, f_hat)
        loss_f10 = self.loss_function(f10, f_hat)
        loss_f11 = self.loss_function(f11, f_hat)
        loss_f12 = self.loss_function(f12, f_hat)
        
        loss_f = loss_f1 + loss_f2 + loss_f3 + loss_f4 + loss_f5 + loss_f6 + loss_f7 + loss_f8 + loss_f9 + loss_f10 + loss_f11 + loss_f12
        
        return loss_f
    
    def loss_vector(self, X_train_Nf):
        g = X_train_Nf.clone()
        g.requires_grad = True
        
        x = self.forward1(g)
        y = self.forward2(g)
        z = self.forward3(g)
        
        loss_v1 = 1 - torch.sqrt(torch.pow(x[:, 1:2], 2) + torch.pow(y[:, 1:2], 2) + torch.pow(z[:, 1:2], 2) + 1e-10)
        loss_v2 = 1 - torch.sqrt(torch.pow(x[:, 2:3], 2) + torch.pow(y[:, 2:3], 2) + torch.pow(z[:, 2:3], 2) + 1e-10)
        loss_v3 = 1 - torch.sqrt(torch.pow(x[:, 3:4], 2) + torch.pow(y[:, 3:4], 2) + torch.pow(z[:, 3:4], 2) + 1e-10)
        
        loss_f13 = self.loss_function(loss_v1, f_hat)
        loss_f14 = self.loss_function(loss_v2, f_hat)
        loss_f15 = self.loss_function(loss_v3, f_hat)
    
        loss_v = loss_f13 + loss_f14 + loss_f15
        
        return loss_v
    
    def loss(self, X_train_Nu, U_train_Nu, X_train_Nf, X_data, U_data):
        loss_u = self.loss_BC(X_train_Nu, U_train_Nu)
        loss_f = self.loss_ODE(X_train_Nf)
        loss_d = self.loss_data(X_data, U_data)
        loss_v = self.loss_vector(X_train_Nf)
        
        loss_val = 10 * loss_u + loss_f + loss_d + 4 * loss_v
        
        return loss_val
    
    def closure(self):
        optimizer.zero_grad()  
        loss = self.loss(X_train_Nu, U_train_Nu, X_train_Nf, X_data, U_data)
        loss.backward()      
        self.iter += 1
        if self.iter % 1000 == 0:
            print("Training Error:",loss.detach().cpu().numpy())
        
        return loss 
    


In [ ]:
from pyDOE import lhs 
ub = np.array([9.65])
lb = np.array([0])

X_train_Nu = np.array([0]).reshape((1, 1))
X_train_Nf = lb + (ub - lb)*lhs(1, N_f) 
X_train_Nf = np.vstack((X_train_Nf, X_train_Nu))
U_train_Nu = np.array([0, 0, 0]).reshape((1, 3))
X_train_Nt = lb + (ub - lb) * lhs(1, N_t)


data = pd.read_excel('data1.xlsx', header = None).to_numpy()
X_data = data[:, 0].reshape((11, 1))
U_data = data[:, 1:4].reshape((11, 3))
print(X_data.shape)
print(U_data.shape)
print(X_train_Nf.shape)

In [ ]:
X_train_Nf = torch.from_numpy(X_train_Nf).float().to(device)
X_train_Nu = torch.from_numpy(X_train_Nu).float().to(device)
U_train_Nu = torch.from_numpy(U_train_Nu).float().to(device)
X_train_Nt = torch.from_numpy(X_train_Nt).float().to(device)
f_hat = torch.zeros(X_train_Nf.shape[0], 1).to(device)
f_hat_t = torch.zeros(X_train_Nt.shape[0], 1).to(device)
X_data = torch.from_numpy(X_data).float().to(device)
U_data = torch.from_numpy(U_data).float().to(device)

In [ ]:
X_train_Nf, _ = torch.sort(X_train_Nf, dim = 0)
X_train_Nt, _ = torch.sort(X_train_Nt, dim = 0)
print(X_train_Nf.shape)
print(X_train_Nu.shape)
print(U_train_Nu.shape)

In [ ]:
springbackcurve = pd.read_excel('springbackcurve830.xlsx', header = None).to_numpy()
springbackcurve = torch.from_numpy(springbackcurve[:, 0:3]).float().to(device)
p = pd.read_excel('point830.xlsx', header = None).to_numpy()

p = torch.from_numpy(p).float().to(device)
def curve_error(springbackcurve, pre_curve):
    subs = pre_curve - springbackcurve
    disVector = torch.sqrt(torch.pow(subs[:, 0:1], 2) + torch.pow(subs[:, 1:2], 2) + torch.pow(subs[:, 2:3], 2))
    i = torch.arange(1, 363)
    i = i.reshape((362, 1)).to(device)
    error_dis = disVector[1:362, 0:1] / (1.5 * (i[1:362, 0:1] - 1))
    error = torch.sqrt(torch.sum(torch.pow(error_dis, 2)) / disVector.shape[0])
    return error

def ED(springbackcurve, pre_curve):
    subs = pre_curve - springbackcurve
    distance = torch.norm(subs, dim = 1)
    mean_dis = torch.mean(distance)
    return mean_dis

def DTW(springbackcurve, pre_curve):
    m, n = springbackcurve.shape[0], pre_curve.shape[0]
    dp = torch.zeros((m + 1, n + 1))
    dp[0, 1:] = float('inf')
    dp[1:, 0] = float('inf')
    dp[1:, 1:] = torch.cdist(springbackcurve, pre_curve, p = 2)
    dp1 = dp[1:, 1:]
    
    for i in range(m):
        for j in range(n):
            dp1[i, j] += min(dp[i, j], dp[i, j + 1], dp[i + 1, j])
            
    return dp1[-1, -1] / sum(dp1.shape)
    
def LCSS(springbackcurve, pre_curve):
    eps = 1
    delta = 5
    m, n = springbackcurve.shape[0], pre_curve.shape[0]
    dp = torch.zeros((m + 1, n + 1))
    dist = torch.cdist(springbackcurve, pre_curve, p = 2)
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if dist[i - 1, j - 1] <= eps:
                dp[i, j] = dp[i - 1, j - 1] + 1
            else:
                dp[i, j] = max(dp[i - 1, j], dp[i, j - 1])
            if abs(i - j) > delta:
                dp[i, j] = 0
    
    return dp[-1, -1] / m

from frechetdist import frdist
def Frechet(springbackcurve, pre_curve):
    frechet = frdist(springbackcurve, pre_curve)
    return frechet

In [ ]:
PINN = FCN(layers)
PINN.to(device)
print(PINN)
params = list(PINN.parameters())

optimizer = torch.optim.Adam(PINN.parameters(),lr=lr,amsgrad=False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10000, gamma = 0.5)
# print(params)

In [ ]:
min_loss = np.array(0.1)
min_loss = torch.from_numpy(min_loss).float().to(device)
loss_list = []
error_list = []
grad_store = []
error_ED_list = []
error_DTW_list = []
error_LCSS_list = []
error_F_list = []
kapa_list = []
tao_list = []

In [ ]:
for i in range(steps):
    loss = PINN.loss(X_train_Nu, U_train_Nu, X_train_Nf, X_data, U_data)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        PINN.kapa.clamp_(0.002, 0.01)
        PINN.tao.clamp_(0.0008, 0.002)
    
    scheduler.step()
    
    pre_curve_all_x = PINN.forward1(p)
    pre_curve_all_y = PINN.forward2(p)
    pre_curve_all_z = PINN.forward3(p)
    pre_curve_all = torch.hstack((pre_curve_all_x[:, 0:1], pre_curve_all_y[:, 0:1], pre_curve_all_z[:, 0:1]))
    pre_curve = pre_curve_all[:, 0:3]
    error = curve_error(springbackcurve, pre_curve)
    error_list.append(error.item())
    kapa_list.append(kapa.item())
    tao_list.append(tao.item())
    
    #损失函数记录
    if i % 400 == 0:
        pre_curve_c = pre_curve.cpu()
        springbackcurve_c = springbackcurve.cpu()
        error_ED = ED(springbackcurve_c, pre_curve_c)
        error_DTW = DTW(springbackcurve_c, pre_curve_c)
        error_LCSS = LCSS(springbackcurve_c, pre_curve_c)
        error_F = Frechet(springbackcurve_c.detach().numpy(), pre_curve_c.detach().numpy())
        error_ED_list.append(error_ED.item())
        error_DTW_list.append(error_DTW.item())
        error_LCSS_list.append(error_LCSS.item())
        error_F_list.append(error_F)
        
    loss_list.append(loss.item())
    
    if i % 2000 == 0:
        # test = PINN.test(X_train_Nu, U_train_Nu, X_train_Nt, X_data, U_data)
        test_BC = PINN.loss_BC(X_train_Nu, U_train_Nu)
        print(" loss: ", loss.detach().cpu().numpy(), "  test_BC loss: ", test_BC.detach().cpu().numpy(), "  error: ", error.detach().cpu().numpy())
        print(" error_ED is: ", error_ED, " error_DTW is: ", error_DTW, " error_LCSS is: ", error_LCSS, " error_F is: ", error_F)
        print("kapa: ", PINN.kapa, "  tao: ", PINN.tao)
        print('**********************************************************************************************' + str(i))
    
    if loss < min_loss:
        min_loss = loss

        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        torch.save(PINN.state_dict(), 'modelzi.pth')
    

In [ ]:
import matplotlib.pyplot as plt
x = PINN.forward1(X_train_Nf) #[1001, 4]
y = PINN.forward2(X_train_Nf)
z = PINN.forward3(X_train_Nf)
test = torch.hstack((x[:, 0:1], y[:, 0:1], z[:, 0:1]))

In [ ]:
plt.plot(X_train_Nf.detach().cpu().numpy(), x[:, 3].detach().cpu().numpy())

In [ ]:
b = test.detach().cpu().numpy()
c = springbackcurve.detach().cpu().numpy()
fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.plot3D(b[:, 0:1], b[:, 1:2], b[:, 2:3], 'red')
ax.plot3D(c[:, 0:1], c[:, 1:2], c[:, 2:3], 'blue')
plt.show()